In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle

path = '/Users/nemo/hku/mfin7036/nlp-ferrari'

### exclude those earning call which is not valid

In [4]:
articles = pd.read_csv(path + '/data/locations.csv')
articles = articles[articles['Children'] == False]
articles = articles[articles['Company'] != 'WMT']
articles = articles[articles['Company'] != 'GOOGL']
articles.dropna(inplace=True)

articles.sort_values(by=['Company', 'Date'], inplace=True)
articles.reset_index(drop=True, inplace=True)

In [5]:
articles_arr = pickle.load(open(path + '/data/articles_arr.p', 'rb'))

In [41]:
no_stops = pickle.load(open(path + '/data/no_stops_words.p', 'rb'))
articles_arr = [' '.join(w) for w in no_stops]

In [66]:
v = TfidfVectorizer(max_df=0.9)
tfidf = v.fit_transform(articles_arr)

In [67]:
tfidf_df = pd.DataFrame(tfidf.toarray(),columns=v.get_feature_names_out())
tfidf_df.to_csv(path + '/data/tfidf.csv')

In [74]:
price_chg_3m = pd.read_csv(path + '/data/price_chg_3m.csv', index_col=0)

## calculate the cosine similarity for groups splitted by stock return

In [122]:
price_chg_3m['p_c'].quantile([0.1, 0.25, 0.3333, 0.5, 0.6666, 0.75, 0.9])

0.1000   -0.090225
0.2500   -0.021786
0.3333    0.003267
0.5000    0.046141
0.6666    0.091554
0.7500    0.117090
0.9000    0.193576
Name: p_c, dtype: float64

In [123]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.193576].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= -0.090225].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11075460358153569
dd 0.10697891406828433
ud 0.10864772492695547


In [108]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.117090].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= -0.021786].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.10802484943470686
dd 0.11128609168985125
ud 0.10886081510936475


In [120]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.091554].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] <= 0.003267].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11019258740489096
dd 0.11370992643594001
ud 0.11124588101567941


In [124]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0.046141].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] < 0.046141].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.1116162053577416
dd 0.11423791783669547
ud 0.11242426038903179


In [121]:
up_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] >= 0].index]
down_tfidf = tfidf_df.iloc[price_chg_3m[price_chg_3m['p_c'] < 0].index]

t = cosine_similarity(up_tfidf, up_tfidf)
print('uu', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(down_tfidf, down_tfidf)
print('dd', (t.sum() - len(t)) / len(t) / (len(t) - 1))

t = cosine_similarity(up_tfidf, down_tfidf)
print('ud', t.mean())

uu 0.11235802826707979
dd 0.11346503567339367
ud 0.11282562386262464
